In [5]:
# Import relevant functionality
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama
from tools.visit_web_page_tool import VisitWebPageSyncTool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool

In [6]:
# Create the agent
memory = MemorySaver()

In [7]:
# Create Ollama model
model = ChatOllama(
    model = "qwen2.5-coder",
    temperature = 0.8,
    num_predict = 256,
    base_url = "http://10.0.0.147:11434"
)

In [8]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [9]:
# documents = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200
# ).split_text(docs)
# vector = Chroma.from_documents(documents=documents, embedding=embeddings)
# retriever = vector.as_retriever(search_kwargs={"k": 5})
# retriever_tool = create_retriever_tool(
#     retriever,
#     "transcript_search",
#     "Search for information about podcasts. For any questions about podcast, you must use this tool!",
# )

In [10]:
agent_executor = create_react_agent(
    model=model,
    tools=[VisitWebPageSyncTool()],
    checkpointer=memory)

In [11]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="what is the weather today use visit_website to pull weather from a website?")],},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

what is the weather today use visit_website to pull weather from a website?
================================== Ai Message ==================================

```json
{"name":"visit_webpage","arguments":{"url":"https://www.weather.com/weather/today"}}
```
